In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import torch
from model import *
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
aita_raw_df = pd.read_csv('aita_clean.csv')
aita_raw_df

,id,timestamp,title,body,edited,verdict,score,num_comments,is_asshole
0,1ytxov,1.393279e+09,[AITA] I wrote an explanation in TIL and came ...,[Here is the post in question](http://www.redd...,False,asshole,52,13.0,1
1,1yu29c,1.393281e+09,[AITA] Threw my parent's donuts away,"My parents are diabetic, morbidly obese, and a...",1393290576.0,asshole,140,27.0,1
2,1yu8hi,1.393285e+09,I told a goth girl she looked like a clown.,I was four.,False,not the asshole,74,15.0,0
3,1yuc78,1.393287e+09,[AItA]: Argument I had with another redditor i...,http://www.reddit.com/r/HIMYM/comments/1vvfkq/...,1393286962.0,everyone sucks,22,3.0,1
4,1yueqb,1.393288e+09,[AITA] I let my story get a little long and bo...,NaN,False,not the asshole,6,4.0,0
...,...,...,...,...,...,...,...,...,...
97623,ex94w5,1.580577e+09,AITA for telling my sister she is being a spoi...,My sister(17F) and I(15M) are white kids born ...,1580585457.0,not the asshole,16,23.0,0
97624,ex970f,1.580577e+09,AITA for telling my husband to f* off after he...,My husband (28M) and I (32F) are married for a...,1580584475.0,not the asshole,1373,304.0,0
97625,ex9dwo,1.580578e+09,AITA for attempting to keep my students out of...,Upfront apologies for formatting. I’m also try...,False,not the asshole,4,15.0,0
97626,ex9egs,1.580578e+09,WIBTA if I left my brothers fate up to the state?,A little back story my mom is a drug addict an...,False,not the asshole,280,140.0,0


In [8]:
aita_raw_df["post"] = aita_raw_df["title"].astype(str) + aita_raw_df["body"].astype(str)
aita_raw_df["post"][0]

'[AITA] I wrote an explanation in TIL and came off as condescending[Here is the post in question](http://www.reddit.com/r/todayilearned/comments/1y40m9/til_a_circle_of_infinite_circumference_is_a/cfhhajs?context=3)\n\nIt seems like I came off as an asshole. What can I do in the future to avoid this? I thought I was just randomly explaining something, which seems common on Reddit.\n\nI definitely shouldn\'t have said "Sorry for trying to help" - That was pretty immature and rude. \n\nAm I the asshole here?\n\nI wrote as little as possible to skew perspective as little as possible.'

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    aita_raw_df.post.values, 
    aita_raw_df.is_asshole.values,
    stratify = aita_raw_df.is_asshole.values,
    test_size = 0.2, 
    random_state = 448
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train,
    stratify = y_train,
    test_size = 0.2, 
    random_state = 448
)
print(X_train[0])
print(y_train[0])

AITA for wanting to dump a girl after finding out she has an STD?So I (32m) started dating a new girl (26f) after a disgusting horrible divorce from my wife who I’ve been with for like 15 years.. 

So I was super patent and deliberate in choosing a girl to date, turning down lots of others over the last year or so of separation... Then I met this girl and she is like EVERYTHING I want in a woman, and I could really see myself having a long-term relationship with her. 

We took things slow and waited about a month to have sex, just fooling around and doing other stuff. Then when the time came she stopped me right before and told me she had been living with herpes since the age of 19.. She was very mature and responsible and understanding and said she’d give me time to think about it, but...TBH I’m fucking freaking out. I’m terrified that this could affect me and my life forever if I end up being with her.

I basically told her all of this, and broke up with her. I said that I couldn’t r

In [11]:
embed = KeyedVectors.load('glove300.kv')

In [18]:
train_data = AITADataset(X_train, y_train, embed)
dev_data = AITADataset(X_val, y_val, embed)
test_data = AITADataset(X_test, y_test, embed)


KeyboardInterrupt



In [ ]:
train_loader = DataLoader(train_data, batch_size=32, collate_fn=basic_collate_fn, shuffle=True)
dev_loader = DataLoader(dev_data, batch_size=32, collate_fn=basic_collate_fn, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, collate_fn=basic_collate_fn, shuffle=False)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# test on validation to see if overfit is possible
net = ensembleCNNBiGRU(256, 256, device)
net.to(device)
optim = get_optimizer(net, lr=1e-3, weight_decay=0)
best_model, stats = train_model(net, dev_loader, dev_loader, optim,
                                num_epoch=100, collect_cycle=20, device=device, patience=None)
plot_loss(stats)

In [ ]:
import itertools
from tqdm.notebook import tqdm

# Credit for this and a bunch of the training/validation code to EECS 487 staff
def search_param():
    """Experiemnt on different hyper parameters."""
    hidden_dim, learning_rate, weight_decay, window_sizes = get_hyper_parameters()
    print("hidden dimension from: {}\nlearning rate from: {}\nweight_decay from: {}\nwindow from: {}".format(
        hidden_dim, learning_rate, weight_decay, window_sizes
    ))
    best_model, best_stats = None, None
    best_accuracy, best_lr, best_wd, best_hd, best_window_size = 0, 0, 0, 0, 0
    for hd, lr, wd, window_size in tqdm(itertools.product(hidden_dim, learning_rate, weight_decay, window_sizes),
                           total=len(hidden_dim) * len(learning_rate) * len(weight_decay) * len(window_sizes)):
        net = FFNN(hd, 200, False, 3, device).to(device)
        optim = get_optimizer(net, lr=lr, weight_decay=wd)
        model, stats = train_model(net, train_loader, dev_loader, optim, num_epoch=100,
                                   collect_cycle=500, device=device, verbose=True, patience=15)
        # print accuracy
        print(f"{(hd, lr, wd, window_size)}: {stats['accuracy']}")
        # update best parameters if needed
        if stats['accuracy'] > best_accuracy:
            best_accuracy = stats['accuracy']
            best_model, best_stats = model, stats
            best_hd, best_lr, best_wd, best_window_size = hd, lr, wd, window_size
    print("\n\nBest hidden dimension: {}, Best learning rate: {}, best weight_decay: {}, best window: {}".format(
        best_hd, best_lr, best_wd, best_window_size))
    print("Accuracy: {:.4f}".format(best_accuracy))
    plot_loss(best_stats)
    return best_model
basic_model = search_param()